# Text generation using LSTM

In [15]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

%matplotlinb inline

UsageError: Line magic function `%matplotlinb` not found.


In [3]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
raw_text[:100]

"alice's adventures in wonderland\n\nlewis carroll\n\nthe millennium fulcrum edition 3.0\n\n\n\n\nchapter i. d"

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
print("Chars:\n",chars)
print("int_to_char:\n",int_to_char)

Chars:
 ['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '0', '3', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
int_to_char:
 {0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: '*', 8: ',', 9: '-', 10: '.', 11: '0', 12: '3', 13: ':', 14: ';', 15: '?', 16: '[', 17: ']', 18: '_', 19: 'a', 20: 'b', 21: 'c', 22: 'd', 23: 'e', 24: 'f', 25: 'g', 26: 'h', 27: 'i', 28: 'j', 29: 'k', 30: 'l', 31: 'm', 32: 'n', 33: 'o', 34: 'p', 35: 'q', 36: 'r', 37: 's', 38: 't', 39: 'u', 40: 'v', 41: 'w', 42: 'x', 43: 'y', 44: 'z'}


In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  144409
Total Vocab:  45


In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
print("dataX[:5]\n", dataX[:5])
print("dataY[:5]\n", dataY[:5])

Total Patterns:  144309
dataX[:5]
 [[19, 30, 27, 21, 23, 4, 37, 1, 19, 22, 40, 23, 32, 38, 39, 36, 23, 37, 1, 27, 32, 1, 41, 33, 32, 22, 23, 36, 30, 19, 32, 22, 0, 0, 30, 23, 41, 27, 37, 1, 21, 19, 36, 36, 33, 30, 30, 0, 0, 38, 26, 23, 1, 31, 27, 30, 30, 23, 32, 32, 27, 39, 31, 1, 24, 39, 30, 21, 36, 39, 31, 1, 23, 22, 27, 38, 27, 33, 32, 1, 12, 10, 11, 0, 0, 0, 0, 0, 21, 26, 19, 34, 38, 23, 36, 1, 27, 10, 1, 22], [30, 27, 21, 23, 4, 37, 1, 19, 22, 40, 23, 32, 38, 39, 36, 23, 37, 1, 27, 32, 1, 41, 33, 32, 22, 23, 36, 30, 19, 32, 22, 0, 0, 30, 23, 41, 27, 37, 1, 21, 19, 36, 36, 33, 30, 30, 0, 0, 38, 26, 23, 1, 31, 27, 30, 30, 23, 32, 32, 27, 39, 31, 1, 24, 39, 30, 21, 36, 39, 31, 1, 23, 22, 27, 38, 27, 33, 32, 1, 12, 10, 11, 0, 0, 0, 0, 0, 21, 26, 19, 34, 38, 23, 36, 1, 27, 10, 1, 22, 33], [27, 21, 23, 4, 37, 1, 19, 22, 40, 23, 32, 38, 39, 36, 23, 37, 1, 27, 32, 1, 41, 33, 32, 22, 23, 36, 30, 19, 32, 22, 0, 0, 30, 23, 41, 27, 37, 1, 21, 19, 36, 36, 33, 30, 30, 0, 0, 38, 26, 23, 1, 31, 2

In [7]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print("X[:2]:\n", X[:2])
print("y[:2]:\n", y[:2])

X[:2]:
 [[[0.42222222]
  [0.66666667]
  [0.6       ]
  [0.46666667]
  [0.51111111]
  [0.08888889]
  [0.82222222]
  [0.02222222]
  [0.42222222]
  [0.48888889]
  [0.88888889]
  [0.51111111]
  [0.71111111]
  [0.84444444]
  [0.86666667]
  [0.8       ]
  [0.51111111]
  [0.82222222]
  [0.02222222]
  [0.6       ]
  [0.71111111]
  [0.02222222]
  [0.91111111]
  [0.73333333]
  [0.71111111]
  [0.48888889]
  [0.51111111]
  [0.8       ]
  [0.66666667]
  [0.42222222]
  [0.71111111]
  [0.48888889]
  [0.        ]
  [0.        ]
  [0.66666667]
  [0.51111111]
  [0.91111111]
  [0.6       ]
  [0.82222222]
  [0.02222222]
  [0.46666667]
  [0.42222222]
  [0.8       ]
  [0.8       ]
  [0.73333333]
  [0.66666667]
  [0.66666667]
  [0.        ]
  [0.        ]
  [0.84444444]
  [0.57777778]
  [0.51111111]
  [0.02222222]
  [0.68888889]
  [0.6       ]
  [0.66666667]
  [0.66666667]
  [0.51111111]
  [0.71111111]
  [0.71111111]
  [0.6       ]
  [0.86666667]
  [0.68888889]
  [0.02222222]
  [0.53333333]
  [0.86666667]
  

In [11]:
# define the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=512, callbacks=callbacks_list)

Epoch 1/50
144309/144309 [==============================] - 85s 592us/step - loss: 3.0871

Epoch 00001: loss improved from inf to 3.08709, saving model to weights-improvement-01-3.0871-bigger.hdf5
Epoch 2/50
144309/144309 [==============================] - 84s 585us/step - loss: 2.9522

Epoch 00002: loss improved from 3.08709 to 2.95223, saving model to weights-improvement-02-2.9522-bigger.hdf5
Epoch 3/50
144309/144309 [==============================] - 84s 584us/step - loss: 2.7942

Epoch 00003: loss improved from 2.95223 to 2.79418, saving model to weights-improvement-03-2.7942-bigger.hdf5
Epoch 4/50
144309/144309 [==============================] - 85s 589us/step - loss: 2.7117

Epoch 00004: loss improved from 2.79418 to 2.71167, saving model to weights-improvement-04-2.7117-bigger.hdf5
Epoch 5/50
144309/144309 [==============================] - 85s 588us/step - loss: 2.6481

Epoch 00005: loss improved from 2.71167 to 2.64814, saving model to weights-improvement-05-2.6481-bigger.hdf5

## Generate Text using saved files as model

In [11]:
from keras.models import load_model
model1 = load_model("weights-improvement-50-2.0083-bigger.hdf5")

In [12]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" nearly as large as himself, and this he handed over to the other,
saying, in a solemn tone, 'for the "


We'll ask the model to predict what comes next based off of the random seed, convert the output numbers to characters and then append it to the pattern

In [14]:
# generate  1000 characters
import sys
for i in range(1000):
    # Get the pattern and predict the new char
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model1.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    
    
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    
    # Update the patten by eliminating the first char and adding
    # the predicted char at the end of the the pattern to generate new char
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

 mock turtle so the cook the sabbit to she courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to

text that was generated doesn't make any sense, and it seems to start simply repeating patterns after a little bit. However, the longer you train the network the better the text that is generated will be.

## Generate Text using saved files as model's weight

In [17]:
# define the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
# Add weight to the model and compile it
filename = "weights-improvement-50-2.0083-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" tle. 'seals, turtles, salmon, and so on;
then, when you've cleared all the jelly-fish out of the way "
 in the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to the courd to t